In [ ]:
!pip install langchain langchain-community langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import pandas as pd
import numpy as np
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import chromadb
from langchain_chroma import Chroma
from langchain.retrievers import BM25Retriever
import uuid
from tqdm import tqdm
import ast
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/search engine project/processed_split_0.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448047 entries, 0 to 448046
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   num               448047 non-null  int64 
 1   name              448047 non-null  object
 2   chunks            447122 non-null  object
 3   chunk_embeddings  447122 non-null  object
dtypes: int64(1), object(3)
memory usage: 13.7+ MB


In [ ]:
df.isnull().sum()

num                   0
name                  0
chunks              925
chunk_embeddings    925
dtype: int64

In [ ]:
df.dropna(subset=['chunks', 'chunk_embeddings'], inplace=True)

In [ ]:
df.isnull().sum()

num                 0
name                0
chunks              0
chunk_embeddings    0
dtype: int64

In [ ]:
def ingestion(df):
  """
  Ingests dataframe content into a ChromaDB collection.
  """
  # Initialize Chroma client and collection
  chroma_client = chromadb.PersistentClient(path='/content/drive/MyDrive/search engine project/vector_db')
  collection = chroma_client.get_or_create_collection(name="subtitles_collection")

  # Convert dataframe to documents using DataFrameLoader
  loader = DataFrameLoader(df, page_content_column='chunks')
  documents = loader.load()

  # Extract embeddings and metadata, text from the dataframe
  text = []
  embeddings = df['chunk_embeddings'].tolist()
  metadatas = []
  ids = []

  for i, row in df.iterrows():
    metadatas.append({'filename': row['name'], 'num': row['num']})
    text.append(row['chunks'])
    ids.append(str(uuid.uuid4()))  # Generate and store UUID

  collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=text,
    metadatas=metadatas,
  )

## Ingesting documents in batch of 40000 docs at a time

In [ ]:
from tqdm import tqdm

def batch_ingestion(df, batch_size=40000):
    """
    Ingests dataframe content into a ChromaDB collection in batches.
    """
    num_rows = len(df)
    for i in tqdm(range(0, num_rows, batch_size)):
        df_batch = df[i:min(i+batch_size, num_rows)]
        df_batch['chunk_embeddings'] = df_batch['chunk_embeddings'].apply(ast.literal_eval)
        ingestion(df_batch)

batch_ingestion(df)

## Querying example

In [ ]:
chroma_client = chromadb.PersistentClient(path='/content/drive/MyDrive/search engine project/vector_db')
chroma_client.list_collections()

[Collection(name=subtitles_collection)]

In [ ]:
# example query to see the workings of querying

def hybrid_search(query: str):
  chroma_client = chromadb.PersistentClient(path='/content/drive/MyDrive/search engine project/vector_db')
  collection = chroma_client.get_collection(name="subtitles_collection")

  embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


  langchain_chroma = Chroma(
      client=chroma_client,
      collection_name='subtitles_collection',
      embedding_function= embeddings
  )
  print('langchain creation done')
  vector_retriever = langchain_chroma.as_retriever(search_kwargs={"k": 15})
  print('vector retriever object done')
  result = vector_retriever.get_relevant_documents(query)
  print('semantic result calculated')

  #rerank using bm25 keyword based
  bm25_retriever = BM25Retriever.from_documents(documents=result)
  top_5_docs = bm25_retriever.get_relevant_documents(query, k=5)

  return result, top_5_docs

In [ ]:
vector_result, hybrid_result = hybrid_search('thor')

langchain creation done
vector retriever object done
semantic result calculated


In [ ]:
vector_result

[Document(page_content='and his loving nature did not discriminate\nhe once loved a swashbuckling passionista\nanother time he loved a wolf woman\non a woman wolf\nbut thors one true love was an\noh wait no\njane foster\nbut sadly\nin the battle for love thor lost\nin fact\nhis mum\nhis dad\nand that guy\nand that guy\nand whoever that is\nand heimdall\nand his brother\nand his brother again\nand again\npoor thor had to watch\nand then he said what have i done\nit seemed that everything\nand so he hid his heart\nso it could never be broken again\nbut just because he was done loving\nhe teamed up with\nand set off on some\nhe got in shape\nputting in the hard yards\nturning pains into gains\nand never skipping leg day\nhe went from dad bod to god bod\nbut beneath his god bod\nthere was still still a sad bod\nbecause all of the bods that\ncouldnt hide the pain\nso he gave up his search for love', metadata={'filename': 'thor love and thunder (2022)', 'num': 9182344}),
 Document(page_conte

In [ ]:
for doc in vector_result:
  print(doc.metadata)

{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'the sandman s02 e01 season of mists prologue (2021)', 'num': 9192540}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'the sandman s02 e01 season of mists prologue (2021)', 'num': 9192540}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}


In [ ]:
for doc in hybrid_result:
  print(doc.metadata)

{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}
{'filename': 'thor love and thunder (2022)', 'num': 9182344}


In [ ]:
!pip install rank_bm25